In [1]:
import os
import os.path as op
from glob import iglob
import re
from csv import reader, writer
import glob

from bids import BIDSLayout
from bids.layout import BIDSLayout, parse_file_entities
import pandas as pd
import numpy as np
from scipy.io import loadmat

from datetime import datetime, timedelta

In [20]:
v1test = {}
v1test_pop = {}
for s,i in pop.items(): 
    if i:
        #did any of the subjects with log in visit 2 had log in visit 1 (seems like no)
        v1test[s] = sorted(glob.glob(f'{psychopy_root}/voice{s}/session001_visit001/behavioral/*{task}*.log')) 
    else:
        #did all the subjects with no log in visit 2 have a log in visit 1
        #v1test_pop[s] = os.listdir(f'{psychopy_root}/voice{s}/') #also see if there are other sessions (there aren't)
        v1test_pop[s] = sorted(glob.glob(f'{psychopy_root}/voice{s}/session001_visit001/behavioral/*{task}*.log')) 


In [2]:
#top level where all subject folders/psychopy log files are
psychopy_root = '/nese/mit/group/sig/om_projects/voice/rawData'
task = 'emosent'

sub_list = [856, 859, 860, 867, 872, 873, 875, 877, 891, 
            893, 895, 896, 897, 898, 957, 962, 963, 967, 
            968, 969, 973, 976, 978, 979, 981]
sub_list = [str(s) for s in sub_list]

pop = {s: sorted(glob.glob(f'{psychopy_root}/voice{s}/session001_visit002/behavioral/*{task}*.log')) for s in sub_list}

#sub_num = '856'

#log file with full path sorted so they are sorted by timestamp value

log_files = {}
for s,i in pop.items(): 
    if i:
        #already found ones from visit 2
        log_files[s] = i 
    else:
        #add logs from visit 1
        log_files[s] = sorted(glob.glob(f'{psychopy_root}/voice{s}/session001_visit001/behavioral/*{task}*.log')) 

# sub_num = '980'
# task = 'nwr'
#log_files_full = sorted(glob.glob(f'{psychopy_root}/voice{sub_num}/session001_visit002/behavioral/*{task}*.log'))
#log_files_full = sorted(glob.glob(f'{psychopy_root}/voice{sub_num}/session001_visit001/behavioral/*{task}*.log'))

#list of names of each log file with base path stripped
#log_files = sorted([lo.split('behavioral/')[1] for lo in log_files_full])  

In [4]:
scan_info = {}
for sub_num in sub_list:
    #load scans.tsv file for this subject and filter by task
    sc = pd.read_table(f'../../sub-voice{sub_num}/ses-1/sub-voice{sub_num}_ses-1_scans.tsv')
    scan_info[sub_num] = sc[sc.filename.str.contains(task)]#[['filename', 'acq_time']].reset_index(drop=True)


In [58]:
run_log_match = {}
    
#loop through each scan for this task and subject
for sub_num, sc in scan_info.items():
    ordered_logs = []
    for i,scan in sc.iterrows():

        #get acqusition time of a scan from scans.tsv file
        s_time_format = '%Y-%m-%dT%H:%M:%S.%f'
        s_time_obj = datetime.strptime(scan.acq_time, s_time_format)
        s_time_obj = s_time_obj.replace(microsecond=0) #drop seconds and ms, just comparing by minutes

        #loop through each log file and compare timestamps of log file to scan.tsv acqusition time
        
        for logf in log_files[sub_num]:

            #remove base path
            log = logf.split('behavioral/')[1]

            #just extract timestamp from filename
            log_time = log.split('_voice')[0]
            log_time_format = '%Y%m%d%H%M%S'
            log_time_obj = datetime.strptime(log_time, log_time_format)

            #check if scan.tsv time and log file time are the same
            # to determine if they are from the same run
            # see if they are with 30 seconds
            if -60 <= s_time_obj.timestamp()- log_time_obj.timestamp() <=60:
                ordered_logs.append((logf, parse_file_entities(scan.filename)))
    run_log_match[sub_num] = (ordered_logs)


In [59]:
test_s = '875'
#867 scan1 is 212 from the "wrong"? one that it closest to scan 2, 213 from the "right" on that is scan1
#seems wrong for a bunch of them

In [60]:
run_log_match[test_s]

[('/nese/mit/group/sig/om_projects/voice/rawData/voice875/session001_visit002/behavioral/20150706115628_voice875_S001_T005_R001_emosent.log',
  {'subject': 'voice875',
   'session': '1',
   'task': 'emosent',
   'run': 1,
   'suffix': 'bold',
   'extension': '.nii.gz'}),
 ('/nese/mit/group/sig/om_projects/voice/rawData/voice875/session001_visit002/behavioral/20150706120056_voice875_S001_T005_R002_emosent.log',
  {'subject': 'voice875',
   'session': '1',
   'task': 'emosent',
   'run': 2,
   'suffix': 'bold',
   'extension': '.nii.gz'})]

In [111]:
log_files[test_s]

['/nese/mit/group/sig/om_projects/voice/rawData/voice875/session001_visit002/behavioral/20150706115628_voice875_S001_T005_R001_emosent.log',
 '/nese/mit/group/sig/om_projects/voice/rawData/voice875/session001_visit002/behavioral/20150706120056_voice875_S001_T005_R002_emosent.log']

In [8]:
scan_info[test_s]

,filename,acq_time,operator,randstr
15,func/sub-voice875_ses-1_task-emosent_run-1_bol...,2015-07-06T11:57:20.057500,NaN,f239d838
16,func/sub-voice875_ses-1_task-emosent_run-2_bol...,2015-07-06T12:01:11.667500,NaN,cfe7e4f7


In [97]:
index = 10
scan_info[test_s].acq_time.loc[index]

'2015-07-27T17:47:09.105000'

In [98]:
s_time_obj = datetime.strptime(scan_info[test_s].acq_time.loc[index], s_time_format)
s_time_obj = s_time_obj.replace(microsecond=0) 
s_time_obj#.timestamp()

datetime.datetime(2015, 7, 27, 17, 47, 9)

In [99]:
log_time_obj = datetime.strptime('20150727175041', log_time_format)
log_time_obj#.timestamp()

datetime.datetime(2015, 7, 27, 17, 50, 41)

In [100]:
s_time_obj.timestamp()- log_time_obj.timestamp()

-212.0

In [38]:
-30 <= s_time_obj.timestamp()- log_time_obj.timestamp() <=30

True

,onset,duration,trial_type,sentence,speak_start
0,3.92,4.0,sad,I can't seem to do well on my exams,5.0525
1,7.92,3.99,silent,n/a,9.0523
2,15.91,4.0,happy,What joke could be funnier than that?,17.0425
3,19.91,4.0,happy,That was a blast,21.0423
4,23.91,4.0,silent,n/a,25.0423
5,27.91,3.99,sad,Please forgive me,29.0419
6,31.90,4.0,happy,I highly recommend that professor,33.0318
7,35.90,4.0,happy,Isn't that beautiful?,37.032
8,39.90,4.0,sad,If only I could go back,41.032
9,43.90,3.99,happy,He's always pleasant to be around,45.0319


In [74]:
for pairs in run_log_match.values():
    for match in pairs:
        logf = match[0]
        evf = match[1]

        full_events = []
        with open(logf, encoding='utf-8') as fp:
            conds = []
            onset_timestamp = []
            onsets = []
            sentences = []
            speak_start_timestamp = []
            # time when first volume is acquired
            start = -1

            for line in fp:
                if 'polygon: autoDraw = True' in line:
                    polygon = line #float(re.search("^\d+.\d+", line).group(0))
                    speak_start_timestamp.append(float(re.search("^\d+.\d+", polygon).group(0)))

                # we only care about new trials
                if 'New trial' not in line:
                    continue
                try:
                    # onset time
                    onset = float(re.search("^\d+.\d+", line).group(0))
                    # condition and sentences are coded as blank
                    cond = re.search("(?<=u'affect': u')\w+", line)
                    cond = cond.group(0) if cond else "fixation"
                    if cond == "NULL":
                        cond = "silent"
                    sent = re.search("(?<=sentence_': u(\'|\"))([\w ?!.']+)", line)

                    # TODO: some sentences are being cut off
                    # see test.tsv
                    sent = (sent.group(0) if sent else "n/a").strip().strip("'")
                    # sentences can also consist of only whitespace
                    if not sent:
                        sent = "n/a"

                except Exception as e:
                    print("Caught error", str(e))
                    print(line)
                    continue

                conds.append(cond)
                sentences.append(sent)
        #         if speak_start:
        #             speak_start[-1] = speak_start[-1] - onset
                onset_timestamp.append(onset)
                if not onsets:
                    onsets.append('0')
                    start = onset
                else:
                    onsets.append('%.2f' % (onset - start))
            dur = [float(y) - float(x) for x,y in zip(onsets,onsets[1:])]
            dur.append(np.mean(dur))
            speak_start = np.add([float(on) for on in onsets], np.subtract(speak_start_timestamp, onset_timestamp))
        #events = (onsets, dur, conds, sentences)
        events = pd.DataFrame([onsets, dur, conds, sentences, speak_start]).T
        events.columns = ['onset', 'duration', 'trial_type', 'sentence', 'speak_start']
        #events = events[events.trial_type != 'neutral'].reset_index(drop=True)

        sub = evf['subject']
        ses = evf['session']
        task = evf['task']
        run = int(evf['run'])

        events.to_csv(f'test/sub-{sub}_ses-{ses}_task-{task}_run-{run}_events.tsv', index= False, sep="\t")

    
    #full_events.append(events)

        #dur = [duration] * len(onsets)

In [ ]:
# PROJECT_ROOT = '/om/project/voice/bids/data'
# pheno_root = '/nese/mit/group/sig/om_projects/voice/rawData/voice856/session001_visit002'
# psychopy = f'{pheno_root}/behavioral/20151030121456_voice856_S001_T005_R002_emosent.log'
# audio = f'{pheno_root}/audio/voice856_S001_T005_R002_emosent.wav'

In [5]:
# subj = 'voice856'
# task = 'emosent'
# run = 2
# ses = 1

In [6]:
#cur_ev = glob.glob(f'/nese/mit/group/sig/om_projects/voice/bids/data/sub-{subj}/ses-{ses}/func/sub-{subj}*task-{task}*run-0{run}*events.tsv')
# cur_ev = [f'../../derivatives/events_emosent_112823/sub-{subj}_ses-{ses}_task-{task}_run-0{run}_events.tsv']

In [7]:
# logf = grab_file(subj, task, run, ses)
# get_emosent(logf).head()

,0,1,2,3
0,0,1.1,neutral,Have you seen him?
1,3.86,1.1,sad,I can't seem to do well on my exams
2,7.85,1.1,silent,n/a
3,11.84,1.1,neutral,The airplane is almost full
4,15.85,1.1,happy,What joke could be funnier than that?


In [32]:
re.search("^\d+.\d+", line)

<re.Match object; span=(0, 8), match='224.4750'>

In [95]:
!ls test

fails.txt
second_level
sub-voice856_ses-1_task-emosent_rec-unco_run-2_space-fsLR_den-91k_contrast-speechvsil_effect_size.dscalar.nii
sub-voice856_ses-1_task-emosent_rec-unco_run-2_space-fsLR_den-91k_contrast-speechvsil_effect_variance.dscalar.nii
sub-voice856_ses-1_task-emosent_rec-unco_run-2_space-fsLR_den-91k_contrast-speechvsil_z_score.dscalar.nii
sub-voice856_ses-1_task-emosent_rec-unco_run-3_space-fsLR_den-91k_contrast-speechvsil_effect_size.dscalar.nii
sub-voice856_ses-1_task-emosent_rec-unco_run-3_space-fsLR_den-91k_contrast-speechvsil_effect_variance.dscalar.nii
sub-voice856_ses-1_task-emosent_rec-unco_run-3_space-fsLR_den-91k_contrast-speechvsil_z_score.dscalar.nii
sub-voice856_ses-1_task-emosent_run-2_events.tsv
sub-voice856_ses-1_task-emosent_run-3_events.tsv


In [96]:
pd.read_table('test/sub-voice856_ses-1_task-emosent_run-2_events.tsv').head()

,onset,duration,trial_type,sentence,speak_start
0,0.00,3.88,happy,I really enjoy our family vacations,1.1265
1,3.88,4.00,happy,We had so much fun last night,5.0128
2,7.88,4.00,sad,I'm so sorry for hurting you,9.0129
3,11.88,3.98,silent,NaN,13.0129
4,15.86,4.00,sad,He never listens to me anymore,16.9932


In [97]:
pd.read_table('../../derivatives/events_emosent_112823/sub-voice856_ses-1_task-emosent_run-01_events.tsv').head()

,onset,duration,trial_type,sentence
0,0.00,4,happy,I really enjoy our family vacations
1,3.88,4,happy,We had so much fun last night
2,7.88,4,sad,I'm so sorry for hurting you
3,11.88,4,silent,NaN
4,15.86,4,sad,He never listens to me anymore


In [23]:
full_events[0]

,onset,duration,trial_type,sentences,speak_start
0,0,3.88,happy,I really enjoy our family vacations,1.1265
1,3.88,4.0,happy,We had so much fun last night,5.0128
2,7.88,4.0,sad,I'm so sorry for hurting you,9.0129
3,11.88,3.98,silent,n/a,13.0129
4,15.86,4.0,sad,He never listens to me anymore,16.9932
5,19.86,3.99,sad,I miss the trips we used to take,20.9843
6,23.85,4.0,neutral,I'm on my way to the meeting,24.9825
7,27.85,4.0,happy,You look so excited,28.9825
8,31.85,4.0,neutral,I wonder what that is about,32.9823
9,35.85,4.0,happy,What could be better?,36.9826


In [107]:
onset_timestamp

[39.5709,
 43.4267,
 47.4191,
 51.4112,
 55.4163,
 59.4149,
 63.4142,
 67.414,
 71.4147,
 75.4152,
 79.4079,
 83.4064,
 87.4051,
 91.4038,
 95.4025,
 99.4022,
 103.4025,
 107.403,
 111.4035,
 115.3957,
 119.3943,
 123.393,
 127.3917,
 131.3909,
 135.3906,
 139.3915,
 143.3918,
 147.3924,
 151.3836,
 155.3822,
 159.3807,
 163.3794,
 167.3792,
 171.3794,
 175.38,
 179.3806,
 183.3727,
 187.3652,
 191.3534,
 195.3521,
 199.3513,
 203.351,
 207.3515,
 211.3521,
 215.3529,
 219.3534,
 223.354]

In [104]:
speak_start[-1] = speak_start[-1] - onset

In [91]:
onsets[0:5]

['0', '3.86', '7.85', '11.84', '15.85']

In [89]:
np.subtract(speak_start,[float(o) for o in onsets])

array([40.6923, 40.6924, 40.6936, 40.693 , 40.6983, 40.707 , 40.7066,
       40.7066, 40.7072, 40.7017, 40.7003, 40.6989, 40.7076, 40.7061,
       40.7052, 40.7048, 40.7054, 40.706 , 40.6996, 40.7081, 40.7068,
       40.7054, 40.704 , 40.7036, 40.7036, 40.7042, 40.7049, 40.6973,
       40.7059, 40.7046, 40.7032, 40.7022, 40.7019, 40.7025, 40.7031,
       40.6965, 40.6975, 40.6972, 40.7057, 40.7044, 40.704 , 40.704 ,
       40.7046, 40.7051, 40.7057, 40.7064, 40.695 ])

In [73]:
with open(logf, encoding='utf-8') as fp:
    conds = []
    onsets = []
    sentences = []
    speak_start = []
    # time when first volume is acquired
    start = -1
    a = [l for l in fp if 'polygon: autoDraw = True' in l]

In [60]:
float(re.search("^\d+.\d+", line).group(0))

224.475

In [59]:
line

'224.4750 \tEXP \ttext_3: autoDraw = True\n'

In [58]:
speak_start

[]

In [51]:
np.mean([float(y) - float(x) for x,y in zip(onsets,onsets[1:])])

3.9952173913043483

In [45]:
len(onsets)

47

In [46]:
len(duration)

46

In [41]:
onset

43.4267

In [6]:
def get_emosent(logf, duration='1.1', outfile=None):
    """Parse Emotional Sentences log file and return onset/condition/duration/sentence"""
    
    with open(logf, encoding='utf-8') as fp:
        conds = []
        onsets = []
        sentences = []
        # time when first volume is acquired
        start = -1

        for line in fp:
            # we only care about new trials
            if 'New trial' not in line:
                continue
            try:
                # onset time
                onset = float(re.search("^\d+.\d+", line).group(0))
                # condition and sentences are coded as blank
                cond = re.search("(?<=u'affect': u')\w+", line)
                cond = cond.group(0) if cond else "fixation"
                if cond == "NULL":
                    cond = "silent"
                sent = re.search("(?<=sentence_': u(\'|\"))([\w ?!.']+)", line)
                # TODO: some sentences are being cut off
                # see test.tsv
                sent = (sent.group(0) if sent else "n/a").strip().strip("'")
                # sentences can also consist of only whitespace
                if not sent:
                    sent = "n/a"

            except Exception as e:
                print("Caught error", str(e))
                print(line)
                continue

            conds.append(cond)
            sentences.append(sent)
            if not onsets:
                onsets.append('0')
                start = onset
            else:
                onsets.append('%.2f' % (onset - start))
            
        dur = [duration] * len(onsets)
        
    #events = (onsets, dur, conds, sentences)
    events = pd.DataFrame([onsets, dur, conds, sentences]).T
    if outfile:
        write_events(events, outfile, ['sentences'])
    return events




def grab_file(subj, task, run, session=1, ext='log'):
    #template = "/om/project/voice/rawData/{subj}/session00{ses}_visi*/behavioral/"
    template = '/nese/mit/group/sig/om_projects/voice/rawData/{subj}/session00{ses}_visi*/behavioral/'
    
    #we dont hit this
    if ext == 'mat':
        template += "*_run{run}.{ext}"
    
    #grabs the pyschopy log file
    else:
        template += "*_R00{run}_{task}*{ext}"
    template = template.format(subj=subj, ses=session, run=run, task=task, ext=ext)
    for fl in iglob(template):
        return fl

def test_emosent():
    ons, dur, con, sen = get_emosent(grab_file('voice969', 'emosent', 1))
    assert all(ons)
    assert all(dur)
    assert all(con)
    assert all(sen)

In [ ]:
# for subj in layout.get_subjects():
#     for ses in layout.get_sessions():
        
#         fls = layout.get(
#                 subject=subj, 
#                 session=ses, 
#                 task='emosent', 
#                 suffix='bold', 
#                 extensions=['nii.gz']
#         )
        
#         # too many or too few runs
#         if not 0 < len(fls) < 3:
#             print("Skipping %s-%s" % (subj, ses))
#             continue
        
#         print("Processing %s-%s" % (subj, ses))
#         for fl in fls:
#             # find matching rawdata
#             raw_fl = grab_file(subj, fl.task, fl.run, ses)
#             if not raw_fl:
#                 continue
            
#             event_fl = op.join(fl.dirname, fl.filename.replace('bold.nii.gz', 'events.tsv'))
#             events = get_emosent(raw_fl, outfile=event_fl)

In [20]:

PROJECT_ROOT = '/om/project/voice/bids/data'


def write_events(events, outfile, xfields=None):
    fields = ['onset', 'duration', 'trial_type']
    if isinstance(xfields, list):
        fields += xfields

    with open(outfile, 'w') as tsv:
        tsvwriter = writer(tsv, delimiter='\t')
        tsvwriter.writerow(fields)
        for i in range(len(events[0])):
            tsvwriter.writerow([ev[i] for ev in events])    
    return outfile


def get_pataka(logf, duration='5.0', outfile=None):
    """Parse PaTaKa (pataka) log file and return onsets/duration/conditions"""
    
    with open(logf) as fp:
        conds = []
        onsets = []
        # time when first volume is acquired
        start = -1

        for line in fp:
            # we only care about new trials
            if 'New trial' not in line:
                continue
            try:
                # condition type
                cond = re.search("(?<=u'instr': u')\w+", line).group(0)
                # onset time
                onset = float(re.search("^\d+.\d+", line).group(0))
            except Exception as e:
                print("Caught error", str(e))
                continue

            conds.append(cond)
            if start == -1:
                onsets.append('0')
                start = onset
            else:
                onsets.append('%.2f' % (onset - start))

    durs = [duration] * len(onsets)
    events = (onsets, durs, conds)
    if outfile:
        write_events(events, outfile)
    return events


def get_pitch_sentences(logf, duration='1.1', write=False):
    """Parse Sentences (pitchsent) log file and return onsets/durations/conditions/sentence"""
    
    with open(logf, encoding='utf-8') as fp:
        conds = []
        onsets = []
        sentences = []
        # time when first volume is acquired
        start = -1

        for line in fp:
            # we only care about new trials
            if 'New trial' not in line:
                continue
            try:
                # onset time
                onset = float(re.search("^\d+.\d+", line).group(0))
                # condition and sentences are coded as blank
                cond = re.search("(?<=u'pitch': u')\w+", line)
                cond = cond.group(0) if cond else "silent"
                sent = re.search("(?<=sentence': u(\'|\"))([\w ?!.']+)", line)
                sent = (sent.group(0) if sent else "n/a").strip().strip("'")
                # sentences can also consist of only whitespace
                if not sent:
                    sent = "n/a"

            except Exception as e:
                print("Caught error", str(e))
                print(line)
                continue

            # do not write missing sentences
            if sent == 'n/a':
                continue
            conds.append(cond)
            sentences.append(sent)
            if not onsets:
                onsets.append('0')
                start = onset
            else:
                onsets.append('%.2f' % (onset - start))
            
        dur = [duration] * len(onsets)
    return onsets, dur, conds, sentences


def get_nwr(csvf, duration='1.1', write=False):
    """Parse Nonword repetition (nwr) CSV file and return onset/duration/condition/word"""
    df = pd.read_csv(csvf)
    
    # extract when first volume is collected
    start = df.loc[:, 'wordonset'][0]
    
    # adjust for late start
    df['wordonset'] = df['wordonset'] - start
    
    # replace Silent with n/a
    # TODO: drop Silent instead
    df['Word'].replace(to_replace="Silent",
                       value="n/a",
                       inplace=True)
    
    dur = [duration] * len(df['wordonset'].tolist())
    
    # round onsets and convert to string
    df = df.round({'wordonset': 2}).astype({'wordonset': str})

    return df['wordonset'].tolist(), dur, df['Syllables'].tolist(), df['Word'].tolist()


def get_face(logf, duration='1.1', outfile=None):
    """Parse face mat file and return onset/condition/duration"""
    
    def get_val(el):
        """Recursive traversal of element until returns a single value"""
        if not hasattr(el, '__iter__') or isinstance(el, str):
            return el
        try:
            el[0]
        except TypeError:
            return el
        return get_val(el[0])
    
    conds = []
    onsets = []
    
    mat = loadmat(logf)
    data = mat['runData']
    assert data is not None

    for val in data:
        conds.append(get_val(val[1]))
        onsets.append('%.2f' % get_val(val[2]))
        
    durs = [duration] * len(onsets)
    events = (onsets, durs, conds)
    if outfile:
        write_events(events, outfile)
    return events


def get_emosent(logf, duration='1.1', outfile=None):
    """Parse Emotional Sentences log file and return onset/condition/duration/sentence"""
    
    with open(logf, encoding='utf-8') as fp:
        conds = []
        onsets = []
        sentences = []
        # time when first volume is acquired
        start = -1

        for line in fp:
            # we only care about new trials
            if 'New trial' not in line:
                continue
            try:
                # onset time
                onset = float(re.search("^\d+.\d+", line).group(0))
                # condition and sentences are coded as blank
                cond = re.search("(?<=u'affect': u')\w+", line)
                cond = cond.group(0) if cond else "fixation"
                if cond == "NULL":
                    cond = "silent"
                sent = re.search("(?<=sentence_': u(\'|\"))([\w ?!.']+)", line)
                # TODO: some sentences are being cut off
                # see test.tsv
                sent = (sent.group(0) if sent else "n/a").strip().strip("'")
                # sentences can also consist of only whitespace
                if not sent:
                    sent = "n/a"

            except Exception as e:
                print("Caught error", str(e))
                print(line)
                continue

            conds.append(cond)
            sentences.append(sent)
            if not onsets:
                onsets.append('0')
                start = onset
            else:
                onsets.append('%.2f' % (onset - start))
            
        dur = [duration] * len(onsets)
        
    events = (onsets, dur, conds, sentences)
    if outfile:
        write_events(events, outfile, ['sentences'])
    return events


def get_vowel(logf, duration='1.1', outfile=None):
    """Parse vowels and return onset/condition/duration/vowel"""
    with open(logf, encoding='utf-8') as fp:
        conds = []
        onsets = []
        vowels = []
        # time when first volume is acquired
        start = -1

        for line in fp:
            # we only care about new trials
            if 'New trial' not in line:
                continue
            try:
                # onset time
                onset = float(re.search("^\d+.\d+", line).group(0))
                # condition and sentences are coded as blank
                prompt = re.search("(?<=u'vowel': u')[\w ]+", line)
                # prompt is made up of {pitch} {vowel}
                if prompt and prompt.group(0).strip():
                    cond, vowel = prompt.group(0).split()
                else:
                    cond = "silent"
                    vowel = "n/a"

            except Exception as e:
                print("Caught error", str(e))
                print(line)
                continue

            conds.append(cond)
            vowels.append(vowel)
            if start == -1:
                onsets.append('0')
                start = onset
            else:
                onsets.append('%.2f' % (onset - start))
            
        dur = [duration] * len(onsets)
        
    events = (onsets, dur, conds, vowels)
    if outfile:
        write_events(events, outfile, ["vowel"])
    return events


def get_pitchnw(logf, duration='1.1', outfile=None):
    """Parse vowels and return onset/condition/duration/pitch"""

    with open(logf, encoding='utf-8') as fp:
        conds = []
        onsets = []
        words = []
        # time when first volume is acquired
        start = -1

        onset_re = re.compile("^\d+.\d+")
        prompt_re = re.compile("(?<=u'word': u')[\w?!.]+")
        for line in fp:
            # we only care about new trials
            if 'New trial' not in line:
                continue
            try:
                # onset time
                onset = float(onset_re.search(line).group(0))
                # condition and sentences are coded as blank
                prompt = prompt_re.search(line)
                # prompt is made up of {pitch} {vowel}
                if prompt and prompt.group(0).strip():
                    word = prompt.group(0)
                    if word[-1] == ".":
                        cond = "statement"
                    elif word[-1] == "?":
                        cond = "question"
                    elif word[-1] == "!":
                        cond = "exclamation"
                    word = word[:-1]
                else:
                    cond = "silent"
                    word = "n/a"

            except Exception as e:
                print("Caught error", str(e))
                print(line)
                continue

            conds.append(cond)
            words.append(word)
            if start == -1:
                onsets.append('0')
                start = onset
            else:
                onsets.append('%.2f' % (onset - start))
            
        dur = [duration] * len(onsets)
        
    events = (onsets, dur, conds, words)
    if outfile:
        write_events(events, outfile, ["nonword"])
    return events

# TODO: other tasks
# 8 - movie

In [21]:
############### 
#    tests
###############

def grab_file(subj, task, run, session=1, ext='log'):
    template = "/om/project/voice/rawData/{subj}/session00{ses}_visi*/behavioral/"
    if ext == 'mat':
        template += "*_run{run}.{ext}"
    else:
        template += "*_R00{run}_{task}*{ext}"
    template = template.format(subj=subj, ses=session, run=run, task=task, ext=ext)
    for fl in iglob(template):
        return fl


def test_pataka():
    ons, dur, con = get_pataka(grab_file('voice969', 'pataka', 2))
    assert all(ons)
    assert all(dur)
    assert all(con)
    assert len(ons) == len(dur) == len(con)

    
def test_pitch_sentences():
    ons, dur, con, sen = get_pitch_sentences(grab_file('voice969', 'pitchsent', 1))
    assert all(con)
    assert all(dur)
    assert all(ons)
    assert all(sen)
    assert len(ons) == len(dur) == len(con) == len(sen)
    
    for c,s in zip(con,sen):
        if s == "n/a":
            assert c == 'silent'
    

def test_nwr():
    ons, dur, con, wrd = get_nwr(grab_file('voice969', 'nwr', 2, ext='csv'))
    assert all(con)
    assert all(dur)
    assert all(ons)
    assert all(wrd)
    assert len(ons) == len(dur) == len(con) == len(wrd)

    
def test_face():
    ons, dur, con = get_face(grab_file('voice969', 'face', 1, ext='mat'))
    assert all(ons)
    assert all(dur)
    assert all(con)
    assert len(ons) == len(dur) == len(con)
    

def test_emosent():
    ons, dur, con, sen = get_emosent(grab_file('voice969', 'emosent', 1))
    assert all(ons)
    assert all(dur)
    assert all(con)
    assert all(sen)
    
def test_vowel():
    ons, dur, con, snd = get_vowel(grab_file('voice969', 'vowel', 1))
    assert all(ons)
    assert all(dur)
    assert all(con)
    assert all(snd)
    assert len(ons) == len(dur) == len(con) == len(snd)
    
    
def test_pitchnw():
    ons, dur, con, wrd = get_pitchnw(grab_file('voice969', 'pitchnw', 1))
    assert all(ons)
    assert all(dur)
    assert all(con)
    assert all(wrd)
    assert len(ons) == len(dur) == len(con) == len(wrd)

test_pataka()
test_pitch_sentences()
test_nwr()
test_face()
test_emosent()
test_vowel()
test_pitchnw()

In [ ]:
layout = BIDSLayout(PROJECT_ROOT, config=['bids'])#, exclude=['derivatives', '.heudiconv', 'code'])

In [26]:
# just grab emosent
# updated for pybids 0.8

for subj in layout.get_subjects():
    for ses in layout.get_sessions():
        
        fls = layout.get(
                subject=subj, 
                session=ses, 
                task='emosent', 
                suffix='bold', 
                extensions=['nii.gz']
        )
        
        # too many or too few runs
        if not 0 < len(fls) < 3:
            print("Skipping %s-%s" % (subj, ses))
            continue
        
        print("Processing %s-%s" % (subj, ses))
        for fl in fls:
            # find matching rawdata
            raw_fl = grab_file(subj, fl.task, fl.run, ses)
            if not raw_fl:
                continue
            
            event_fl = op.join(fl.dirname, fl.filename.replace('bold.nii.gz', 'events.tsv'))
            events = get_emosent(raw_fl, outfile=event_fl)

Processing voice844-1
Skipping voice844-2
Processing voice846-1
Skipping voice846-2
Processing voice849-1
Skipping voice849-2
Processing voice850-1
Skipping voice850-2
Processing voice852-1
Skipping voice852-2
Processing voice854-1
Skipping voice854-2
Processing voice856-1
Skipping voice856-2
Processing voice857-1
Skipping voice857-2
Processing voice859-1
Skipping voice859-2
Processing voice860-1
Skipping voice860-2
Processing voice862-1
Skipping voice862-2
Processing voice863-1
Skipping voice863-2
Processing voice864-1
Skipping voice864-2
Processing voice867-1
Skipping voice867-2
Processing voice872-1
Skipping voice872-2
Processing voice873-1
Skipping voice873-2
Processing voice875-1
Processing voice875-2
Processing voice877-1
Skipping voice877-2
Processing voice880-1
Skipping voice880-2
Processing voice884-1
Skipping voice884-2
Skipping voice886-1
Skipping voice886-2
Processing voice889-1
Processing voice889-2
Processing voice891-1
Skipping voice891-2
Processing voice893-1
Processing

In [57]:
if not 0 < 0 < 4:
    print('yes')

yes


In [73]:
event_fl

'/om/project/voice/bids/data/sub-voice844/ses-1/func/sub-voice844_ses-1_task-emosent_run-01_events.tsv'

In [8]:
x = layout.get(subject='voice969', session=1, task='emosent', suffix='bold', extensions=['nii.gz'])

In [4]:
fl = '/om/project/voice/rawData/voice969/session001_visit002/behavioral/'

In [8]:
from scipy.io import loadmat
import numpy as np

In [15]:
get_pitchnw(grab_file('voice969', 'pitchnw', 1))

(['0',
  '1.88',
  '3.88',
  '5.88',
  '7.88',
  '9.88',
  '11.88',
  '13.88',
  '15.88',
  '17.88',
  '19.88',
  '21.88',
  '23.87',
  '25.86',
  '27.85',
  '29.85',
  '31.85',
  '33.85',
  '35.85',
  '37.86',
  '39.85',
  '41.85',
  '43.85',
  '45.85',
  '47.85',
  '49.85',
  '51.85',
  '53.85',
  '55.85',
  '57.84',
  '59.84',
  '61.83',
  '63.83',
  '65.83',
  '67.82',
  '69.82',
  '71.82',
  '73.82',
  '75.82',
  '77.82',
  '79.82',
  '81.82',
  '83.82',
  '85.82',
  '87.82',
  '89.82',
  '91.82',
  '93.81',
  '95.80',
  '97.80',
  '99.80',
  '101.80',
  '103.80',
  '105.80',
  '107.79',
  '109.80',
  '111.79',
  '113.79',
  '115.79',
  '117.79',
  '119.80',
  '121.79',
  '123.79',
  '125.79',
  '127.78',
  '129.77',
  '131.77',
  '133.77',
  '135.77',
  '137.77',
  '139.77',
  '141.77',
  '143.77',
  '145.77',
  '147.77',
  '149.77',
  '151.77',
  '153.77',
  '155.77',
  '157.77',
  '159.76',
  '161.75',
  '163.74',
  '165.74',
  '167.74',
  '169.74',
  '171.74',
  '173.74',
  '1